# Appendix: Import needed modules 

In [185]:
## Spell checker from Peter Norvig. Not perfect, but works.
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('C:/Users/kennd/Downloads/big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [186]:
## Import all modules needed
## lem is lemmatizing the words
def lem(x):
    sentence = ''
    for x in re.sub('[^a-z0-9 ]','',x).split():
        sentence = sentence + ' ' + lemmatizer.lemmatize(correct(x))
    return sentence

import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=1,stop_words=stops)

import pandas as pd
df = pd.read_csv('C:/Users/kennd/Desktop/yelp_test_kenn_2.csv')

# Download data and munge munge

In [187]:
# df = df.drop(['wait','ambience','food','service','value','none'],axis=1)

In [188]:
# df2 = df.fillna(0)

In [189]:
df2['s_lower'] = df2['sentence'].apply(lambda x: x.lower())

In [190]:
## lemmatizing and spell checking the words
## this  pre-processing of text is needed before feeding it into tokenizing by word
df2['s_spell_lem'] = df2['s_lower'].apply(lem)

In [191]:
df3 = df2[['s_spell_lem','category']]

In [230]:
corp = list(df3.s_spell_lem)

In [231]:
df3

,s_spell_lem,category
0,rocky ha been in business since the 1980s and...,others
1,their food is good,food
2,not haute quinine pretension,food
3,everyday well prepared and taste bud pleasing...,food
4,give mostly gone during supper hour and and a...,food
5,he ordered the large version of fish sandwich,food
6,rocky is famous for their fish,food
7,three large dreaded and fried fish file but a...,value
8,needle to say he had no complaint and brought...,value
9,i ordered the meatal drill,food


# Feature Generation

In [193]:
## Gets the count of each word in each sentence
feature = vectorizer.fit_transform(corp)

In [194]:
## Transform into word counts into pandas data frame
featuredf = pd.DataFrame(feature.A, columns=vectorizer.get_feature_names())

In [195]:
## Crude normalization of features
featuredf2 = featuredf.loc[:,featuredf.sum()>= 3]
featuredf2 = featuredf2.loc[:,featuredf2.sum()<= 20]

In [196]:
df5 = pd.concat((df3,featuredf2),axis=1)

In [197]:
X = df5.iloc[:,2:]
Y = df5.iloc[:,1]

In [232]:
df5

,s_spell_lem,category,also,always,back,bacon,bar,best,better,bread,...,true,wa,wait,well,worth,would,year,sentiment,business_id,star
0,rocky ha been in business since the 1980s and...,others,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fNGIbpazjTRdXgwRY_NIXA,4
1,their food is good,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,fNGIbpazjTRdXgwRY_NIXA,4
2,not haute quinine pretension,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fNGIbpazjTRdXgwRY_NIXA,4
3,everyday well prepared and taste bud pleasing...,food,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2,fNGIbpazjTRdXgwRY_NIXA,4
4,give mostly gone during supper hour and and a...,food,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,fNGIbpazjTRdXgwRY_NIXA,4
5,he ordered the large version of fish sandwich,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fNGIbpazjTRdXgwRY_NIXA,4
6,rocky is famous for their fish,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fNGIbpazjTRdXgwRY_NIXA,4
7,three large dreaded and fried fish file but a...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1,fNGIbpazjTRdXgwRY_NIXA,4
8,needle to say he had no complaint and brought...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1,fNGIbpazjTRdXgwRY_NIXA,4
9,i ordered the meatal drill,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fNGIbpazjTRdXgwRY_NIXA,4


# Building the model and in-training-set accuracy

In [198]:
from sklearn.cross_validation import train_test_split 

In [217]:
##Split into train and test at 75/25
train, test = train_test_split(df5,test_size = 0.25)

In [218]:
##Split X & Y
X_train = train.iloc[:,2:]
Y_train = train.iloc[:,1]
X_test = test.iloc[:,2:]
Y_test = test.iloc[:,1]

In [219]:
## Just used NB because that's what the scikit-learn tutorial shows
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train,Y_train)

In [220]:
## tried SVC because its supposedly very good at classifying text
from sklearn.svm import SVC
svc = SVC(gamma=2).fit(X,Y)

In [221]:
## NB In-training-set accuracy
Y_PRED = clf.predict(X_test)
np.mean(Y_PRED == np.array(Y_test))

0.5

In [222]:
## SVC In-training-set accuracy
Y_PRED = svc.predict(X_test)
np.mean(Y_PRED == np.array(Y_test))

0.91176470588235292

# Sentiment Scoring

In [224]:
## Modified off-the-shelf sentiment analyzer
import nltk.sentiment.util
def demo_liu_hu_lexicon2(sentence, plot=False):
    """
    Basic example of sentiment classification using Liu and Hu opinion lexicon.
    This function simply counts the number of positive, negative and neutral words
    in the sentence and classifies it depending on which polarity is more represented.
    Words that do not appear in the lexicon are considered as neutral.

    :param sentence: a sentence whose polarity has to be classified.
    :param plot: if True, plot a visual representation of the sentence polarity.
    """
    from nltk.corpus import opinion_lexicon
    from nltk.tokenize import treebank

    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence)]

    x = list(range(len(tokenized_sent))) # x axis for the plot
    y = []

    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
            y.append(1) # positive
        elif word in opinion_lexicon.negative():
            neg_words += 1
            y.append(-1) # negative
        else:
            y.append(0) # neutral

    return (pos_words - neg_words)
#     if pos_words > neg_words:
#         return 'Positive'
#     elif pos_words < neg_words:
#         return 'Negative'
#     elif pos_words == neg_words:
#         return 'Neutral'

    if plot == True:
        _show_plot(x, y, x_labels=tokenized_sent, y_labels=['Negative', 'Neutral', 'Positive'])

In [225]:
df5['sentiment'] = df5['s_spell_lem'].apply(lambda x: demo_liu_hu_lexicon2(x))

In [226]:
## Positive values mean positive sentiment, zero for neutral, negative for..negative
df5.head(20)

,s_spell_lem,category,also,always,back,bacon,bar,best,better,bread,...,time,toast,true,wa,wait,well,worth,would,year,sentiment
0,rocky ha been in business since the 1980s and...,others,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,their food is good,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,not haute quinine pretension,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,everyday well prepared and taste bud pleasing...,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2
4,give mostly gone during supper hour and and a...,food,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,he ordered the large version of fish sandwich,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,rocky is famous for their fish,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,three large dreaded and fried fish file but a...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
8,needle to say he had no complaint and brought...,value,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
9,i ordered the meatal drill,food,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [227]:
df5['business_id'] = df['Business ID']
df5['star'] = df['Review stars']

# Processing Data for Visualization

In [228]:
##Sentiments should be normalized, but whatever for this example
df5.groupby(['business_id','category']).sum().loc[:,['sentiment']]

sentiment
business_id            category           
b9WZJp5L1RZr4F1nxclOoQ ambience          3
                       food             32
                       others            7
                       service           6
                       value            -1
                       wait              5
fNGIbpazjTRdXgwRY_NIXA ambience          3
                       food             15
                       others            4
                       service           1
                       value             4

In [229]:
##Interesting to see how 5 stars generally have positive sentiments on all aspects
df5.groupby(['star','category']).sum().loc[:,['sentiment']]

sentiment
star category           
3    ambience          3
     food              6
     others            3
     service           0
     value             4
     wait              0
4    ambience          0
     food             14
     others            3
     service           2
     value            -2
     wait              4
5    ambience          3
     food             27
     others            5
     service           5
     value             1
     wait              1